In [ ]:
# imports
import pandas as pd
from thunderpack import ThunderReader
from tqdm import tqdm
import re
import os

This file should be run once.  It creates the icd partition and discharge csv's in order to free up ram.

# Discharge Summaries

In [ ]:
year = 2013
output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/MGB/CSVs'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

for i in range(0, 12):
    df = pd.read_csv(f'/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_{year + i}_metadata.csv')
    df = df[df['InpatientNoteTypeDSC'] == 'Discharge Summary']
    df.to_csv(os.path.join(output_dir, f'discharge_summaries_{year + i}.csv'), index=False)


In [ ]:
output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/MGB/CSVs'

# Concatenate all filtered CSV files
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.startswith('discharge_summaries_')]
discharge_summaries = pd.concat((pd.read_csv(f) for f in all_files), axis=0, ignore_index=True)

# Show number of total discharge summaries
print(len(discharge_summaries))

# Show example
discharge_summaries.head()

In [ ]:
#filter by unique patient ID
discharged_patients = set(discharge_summaries['BDSPPatientID'])
print(len(discharged_patients))

# ICD Partitions

In [ ]:
reader = ThunderReader('/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_icd_9_10_1m_MGB')
key_length = len(list(reader.keys()))
print(key_length)

In [ ]:
output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/MGB/CSVs'
os.makedirs(output_dir, exist_ok=True)

for i in tqdm(range(1, key_length + 1)):
    df = reader[f'ICD_partition_{i}']
    df = df[df['BDSPPatientID'].isin(discharged_patients)]
    df.to_csv(os.path.join(output_dir, f'filtered_ICD_partition_{i}.csv'), index=False)